In [7]:
import plotly.graph_objects as go
import pandas as pd
# Example data structure (replace with actual data passed in your Flask app)
selected_player='2544'


selected_group='JUMP_SHOTS'
selected_year=2025

avg_df = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/refs/heads/master/shot_types.csv')

# Read player shot data


player_df = pd.read_csv(f'https://raw.githubusercontent.com/gabriel1200/shot_averages/refs/heads/main/{selected_player}.csv')



if selected_player:
    player_df = player_df[player_df['PLAYER_ID'] == int(selected_player)]
 

# Filter by shot group
group_df = player_df[player_df['shot_group'] == selected_group]
shot_type='2PT Field Goal'
group_df=group_df[group_df['shot_type']==shot_type]

avg_df=avg_df[avg_df['shot_type']==shot_type]

# Calculate efficiencies
shot_data = []
chart_data = {'labels': [], 'player_pct': [], 'league_pct': []}

for shot_desc in group_df['shot_description'].unique():
    print(shot_desc)
    player_shots = group_df[group_df['shot_description'] == shot_desc]
    league_avg = avg_df[(avg_df['shot_description'] == shot_desc) & 
                      (avg_df['year'] == int(selected_year))]
    print(league_avg)
    
    if not player_shots.empty and not league_avg.empty:
        player_pct = (player_shots['shot_makes'].sum() / player_shots['shot_count'].sum()) * 100
        league_pct = league_avg['efg%'].iloc[0] * 100
        
        shot_data.append({
            'shot_description': shot_desc,
            'player_pct': player_pct,
            'league_pct': league_pct,
            'difference': player_pct - league_pct,
            'attempts': player_shots['shot_count'].sum()
        })
        
        chart_data['labels'].append(shot_desc)
        chart_data['player_pct'].append(player_pct)
        chart_data['league_pct'].append(league_pct)

# Create the bar traces
player_efficiency_trace = go.Bar(
    name="Player Efficiency",
    x=chart_data["labels"],
    y=chart_data["player_pct"],
    marker=dict(
        color="rgba(96, 165, 250, 0.8)",  # blue-400
        line=dict(color="rgba(96, 165, 250, 1)", width=1)
    )
)

league_average_trace = go.Bar(
    name="League Average",
    x=chart_data["labels"],
    y=chart_data["league_pct"],
    marker=dict(
        color="rgba(249, 115, 22, 0.8)",  # orange-500
        line=dict(color="rgba(249, 115, 22, 1)", width=1)
    )
)

# Create the figure
fig = go.Figure(data=[player_efficiency_trace, league_average_trace])

# Update layout
fig.update_layout(
    title=dict(
        text="Shot Efficiency Comparison",
        font=dict(color="#F3F4F6")  # text-gray-100
    ),
    legend=dict(
        title="Legend",
        font=dict(color="#F3F4F6")  # text-gray-100
    ),
    xaxis=dict(
        title=dict(text="Players", font=dict(color="#F3F4F6")),  # text-gray-100
        tickangle=45,
        tickfont=dict(color="#F3F4F6"),  # text-gray-100
        gridcolor="rgba(107, 114, 128, 0.2)"  # gray-500 with opacity
    ),
    yaxis=dict(
        title=dict(text="Efficiency %", font=dict(color="#F3F4F6")),  # text-gray-100
        tickfont=dict(color="#F3F4F6"),  # text-gray-100
        gridcolor="rgba(107, 114, 128, 0.2)"  # gray-500 with opacity
    ),
    barmode="group",  # Grouped bar chart

    width=1200,
    height=1000,
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
    paper_bgcolor="rgba(0, 0, 0, 0)"  # Transparent background
)

# Render the figure
fig.show()

NameError: name 'pr' is not defined